In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from utils.preprocess import *
from utils.model import *

### Reduced ts

In [5]:
produv_train = pd.read_csv(config['data_path'] + 'produv_train.csv')
gas_train = pd.read_csv(config['data_path'] + 'gas_train.csv')
chronom_train = pd.read_csv(config['data_path'] + 'chronom_train.csv')

produv_test = pd.read_csv(config['data_path'] + 'produv_test.csv')
gas_test = pd.read_csv(config['data_path'] + 'gas_test.csv')
chronom_test = pd.read_csv(config['data_path'] + 'chronom_test.csv')

In [8]:
reduce_ts(ts_preproc(gas_train, 'Time'), chronom_train).to_csv(config['data_path'] + 'reduced_gas_train.csv')
reduce_ts(ts_preproc(produv_train, 'SEC'), chronom_train).to_csv(config['data_path'] + 'reduced_produv_train.csv')
reduce_ts(ts_preproc(gas_test, 'Time'), chronom_test).to_csv(config['data_path'] + 'reduced_gas_test.csv')
reduce_ts(ts_preproc(produv_test, 'SEC'), chronom_test).to_csv(config['data_path'] + 'reduced_produv_test.csv')

SyntaxError: invalid syntax (Temp/ipykernel_8948/3193017951.py, line 4)

### Extract ts features

In [2]:
train_data = read_data('train')

In [3]:
X_train = preprocess_data(train_data)

 55%|████████████████████████████████████████████                                    | 429/780 [00:45<00:37,  9.36it/s]


ValueError: Input signal must have a length >= n_coeff

In [23]:
test_data = read_data('test')
for file in files:
    test_data[file] = test_data[file][test_data[file].NPLV != 512905]

In [24]:
X_test = preprocess_data(test_data)

100%|████████████████████████████████████████████████████████████████████████████████| 779/779 [01:13<00:00, 10.58it/s]


In [29]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

X_train = X_train.select_dtypes(include=numerics)
X_test  = X_test.select_dtypes(include=numerics)

In [2]:
#X_train.to_csv(config['data_path'] + 'num_extracted_features_train.csv')
#X_test.to_csv(config['data_path'] + 'num_extracted_features_test.csv')
#X_train = pd.read_csv(config['data_path'] + 'num_extracted_features_train.csv')
#X_test  = pd.read_csv(config['data_path'] + 'num_extracted_features_test.csv')

In [3]:
train_NPLV = X_train['NPLV']
test_NPLV = X_test['NPLV']
selected_features_train, selected_features_test = ts_select_features(X_train.drop(['NPLV'], axis = 1), X_test.drop(['NPLV'], axis = 1))

In [6]:
512905

512905

In [6]:
selected_features_test = selected_features_test.drop(['Unnamed: 0'], axis = 1)
selected_features_test['NPLV'] = test_NPLV

In [29]:
imposter = pd.DataFrame(selected_features_test.mean(),columns=[779]).T
imposter['NPLV'] = [512905]

In [34]:
selected_features_test = pd.concat([selected_features_test, imposter]).sort_values('NPLV').reset_index().drop(['index'],axis=1)

In [35]:
selected_features_train = selected_features_train.drop(['Unnamed: 0'], axis = 1)
selected_features_train['NPLV'] = train_NPLV

In [36]:
selected_features_train.shape

(2063, 3150)

In [38]:
selected_features_train.to_csv(config['data_path'] + 'preprocessed_train.csv')
selected_features_test.to_csv(config['data_path'] + 'preprocessed_test.csv')